In [0]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from nltk.tokenize import word_tokenize
#from nltk.tokenize import word_tokenize
import re
import nltk
from keras.layers import Embedding, LSTM
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout, Conv2D, MaxPooling2D

import json

sentiment_label = []
test_label=[]

'''
review_list = []
with open('/content/drive/My Drive/dataset/전처리 전 json/tempe_3000.json', 'rt', -1, encoding='UTF-8') as f:
    for line in f:
        lineobj = json.loads(line)
        review_txt = re.sub('[^a-zA-Z ]', '', lineobj['text'])
        review_txt = review_txt.lower()
        review_list.append(review_txt)
        
print(review_list)
            

# test data
test_list = []
with open('/content/drive/My Drive/dataset/전처리 전 json/vegas_3000.json', 'rt', -1, encoding='UTF-8') as f:
    for line in f:
        lineobj = json.loads(line)
        test_txt = re.sub('[^a-zA-Z ]', '', lineobj['text'])
        test_txt = test_txt.lower()
        test_list.append(test_txt)
'''


result = []
review_list = []
with open('/content/drive/My Drive/dataset/token_list/tempe_3000_tokens.txt', 'rt', encoding='UTF-8') as mf:
    while True:
        temp = mf.readline()
        if not temp: break
        if temp == '[\n':
            continue
        elif temp == ']\n':
            review_list.append(result)
            result = []
            continue
        else:
            l = len(temp)
            word = temp[0:l - 1]
            result.append(word)

result = []
test_list = []
with open('/content/drive/My Drive/dataset/token_list/tempe_3000_tokens.txt', 'rt', encoding='UTF-8') as mf:
    while True:
        temp = mf.readline()
        if not temp: break
        if temp == '[\n':
            continue
        elif temp == ']\n':
            test_list.append(result)
            result = []
            continue
        else:
            l = len(temp)
            word = temp[0:l - 1]
            result.append(word)

with open('/content/drive/My Drive/dataset/전처리 전 json/tempe_3000.json', 'rt', -1, encoding='UTF-8') as f:
    for line in f:
        lineobj = json.loads(line)
        star = lineobj['stars']
        if star >= 4:
            sentiment_label.append('2')
        elif star == 3:
            sentiment_label.append('1')
        else:
            sentiment_label.append('0')
            
sentiment_label = array(sentiment_label)

# test data
with open('/content/drive/My Drive/dataset/전처리 전 json/vegas_3000.json', 'rt', -1, encoding='UTF-8') as f:
    for line in f:
        lineobj = json.loads(line)
        star = lineobj['stars']
        if star >= 4:
            test_label.append('2')
        elif star == 3:
            test_label.append('1')
        else:
            test_label.append('0')
            
test_label = array(test_label)

"""           
word_list = []
for i in range(len(result_c)):
  for j in range(len(result_c[i])):
    word_list.append(result_c[i][j])
    
    
print(word_list)
print('******')
print(len(word_list))
unique_words = set(word_list)
print(len(unique_words))  #9422

word_num = len(unique_words)
vocab_length = word_num+5
"""   

#embedded_sentences = [one_hot(sent, vocab_length) for sent in result_c]
#print(embedded_sentences)
"""
embedded_sentences = []
for i in range(len(result_c)):
  for sent in result_c[i]:
    embedded_sentences.append(one_hot(sent, vocab_length))
    embedded_sentences[i].append(temp[i])
print(embedded_sentences)
"""
'''
all_words = []
for sent in review_list:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)
#print(all_words)
unique_words = set(all_words)
print(len(unique_words))
'''
all_words = []
for sent in review_list:
  for w in sent:
    all_words.append(w)
#print(all_words)
unique_words = set(all_words)
#print(len(unique_words))
vocab_length = len(unique_words)+5
'''
# test data
test_words = []
for sent in test_list:
    tokenize_word_test = word_tokenize(sent)
    for word in tokenize_word_test:
        test_words.append(word)
#print(all_words)
unique_words_test = set(test_words)
test_length = len(unique_words_test)+5 # 5 안 더하면 어떤지 확인하기
'''
test_words = []
for sent in test_list:
  for word in sent:
    test_words.append(word)
#print(all_words)
unique_words_test = set(test_words)
test_length = len(unique_words_test)+5 # 5 안 더하면 어떤지 확인하기

embedded_sentences = []
embedded_sentences_test = []
for sent in review_list:
  for w in sent:
    oh = one_hot(w, vocab_length)
    embedded_sentences.append(oh)
    
#print(embedded_sentences)
for sent in test_list:
  for t in sent:
    oh = one_hot(t, vocab_length)
    embedded_sentences_test.append(oh)
    
#embedded_sentences = [one_hot(sent, vocab_length) for sent in review_list]
#embedded_sentences_test = [one_hot(sent, test_length) for sent in test_list]
#print(embedded_sentences)

tok_list = []
tok_test_list = []
for word in review_list:
  for w in word:
    tok_list.append(w)

for word in test_list:
  for w in word:
    tok_test_list.append(w)

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(tok_list, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
#print(padded_sentences)

# test data
word_count_test = lambda sentence_test: len(word_tokenize(sentence_test))
longest_sentence_test = max(tok_test_list, key=word_count_test)
length_long_sentence_test = len(word_tokenize(longest_sentence_test))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
padded_sentences_test = pad_sequences(embedded_sentences_test, length_long_sentence_test, padding='post')

model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
#model.add(Flatten())
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='softmax'))
"""
model = Sequential()
model.add(Embedding(vocab_length, 100, input_length=length_long_sentence))
model.add(Conv2D(filters=128, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2))
model.add(Conv2D(filters=128, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
"""

#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())
result = model.fit(padded_sentences, sentiment_label, epochs=100, batch_size=100 ,verbose=1)

loss, accuracy = model.evaluate(padded_sentences_test, test_label, verbose=1)
#loss, accuracy = model.evaluate(padded_sentences, sentiment_label, verbose=1)
print('Accuracy: %f' % (accuracy*100))

# 5. 학습과정 살펴보기
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(result.history['loss'], 'y', label='train loss')
loss_ax.plot(result.history['val_loss'], 'r', label='val loss')
loss_ax.set_ylim([-0.2, 1.2])

acc_ax.plot(result.history['acc'], 'b', label='train acc')
acc_ax.plot(result.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylim([-0.2, 1.2])

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()


SyntaxError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True